This file is used to generate metadata for the tensorflow model.

In [1]:
# install the dependencies
!pip install tflite-support

In [2]:
# import the requires modules
import os
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb


In [3]:
# specify the required variables, these need to be kept in sync with capstone.ipynb
IMG_WIDTH = 250
IMG_HEIGHT = 250

class_names = ['1','2','3','4','5']
class_names_labels = ['Water', 'Tea Leaves', 'Boiling tea leaves', 'Tea with Milk', 'Boiling tea']

code_loc = 'code/final/kraftwerk/capstone_project'
data_path = fr'{os.getenv("HOME")}/{code_loc}/data/classes'
data_path_unknown = fr'{os.getenv("HOME")}/{code_loc}/data/unclassified'
tflite_model_path = fr'{os.getenv("HOME")}/{code_loc}/assets/model.tflite'
export_model_path = fr'{os.getenv("HOME")}/{code_loc}/model'
assets_path = fr'{os.getenv("HOME")}/{code_loc}/assets/'
export_json_file = os.path.join(assets_path, os.path.splitext(os.path.basename(export_model_path))[0] + ".json")
labelmap_file = fr'{os.getenv("HOME")}/{code_loc}/assets/labelmap.pbtxt'


In [4]:

"""Creates the metadata for an image classifier."""

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "Custom Image classifier 250x250 Object Detector"
model_meta.description = ("Detects different steps in the process of Milk Tea making process.")
model_meta.version = "v1"
model_meta.author = "Pulkit Bhanot"
model_meta.license = ("Apache License. Version 2.0 "
                      "http://www.apache.org/licenses/LICENSE-2.0.")

In [5]:
# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()

# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()


In [6]:
input_meta.name = "image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(IMG_WIDTH, IMG_HEIGHT))
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)

In [7]:
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the 5 labels respectively."
output_meta.content = _metadata_fb.ContentT()
output_meta.content.content_properties = _metadata_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_stats = _metadata_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = _metadata_fb.AssociatedFileT()
label_file.name = os.path.basename(labelmap_file)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]


In [8]:
# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

In [9]:
populator = _metadata.MetadataPopulator.with_model_file(tflite_model_path)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([labelmap_file])
populator.populate()

In [10]:
displayer = _metadata.MetadataDisplayer.with_model_file(tflite_model_path)

json_file = displayer.get_metadata_json()
# Optional: write out the metadata as a json file
with open(export_json_file, "w") as f:
  f.write(json_file)